<a href="https://colab.research.google.com/github/haticecankaya/AutonomousParkingProject/blob/main/otonom_park_dataset_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
num_samples = 15000
np.random.seed(42)

L = 2.5
W_slot = 2.5
L_slot = 5
w_car = 1.8
l_car = 4.0
s_margin = 0.15
k_theta = 0.5  # Açı düzeltme katsayısı
k_y = 0.2      # Yanal düzeltme katsayısı
delta_max = 0.5 # Maksimum direksiyon açısı
a_base = 4.0 # m/s^2 kuru zemin ivmesi

In [ ]:
side_offset = np.random.uniform(-0.25, 0.25, num_samples)
d_r = ((W_slot - w_car) / 2) - side_offset
d_l = ((W_slot - w_car) / 2) + side_offset
long_pos = np.random.uniform(0.1, 0.7, num_samples)
d_f = long_pos
d_b = L_slot - l_car - long_pos

In [ ]:
data = {
    'id': np.arange(1, num_samples + 1),
    'r': np.random.uniform(0, 1, num_samples),
    'd_f': d_f,
    'd_b': d_b,
    'd_r': d_r,
    'd_l': d_l,
    'v': np.random.uniform(-0.3, 1.5, num_samples),
    'e_lane': side_offset + np.random.normal(0, 0.02, num_samples),
    'conf_lane': np.random.uniform(0.7, 1.0, num_samples),
    'batt': np.random.uniform(0.1, 0.9, num_samples),
    'δ': np.random.uniform(-0.5, 0.5, num_samples),
    'e_θ': np.random.uniform(-0.2, 0.2, num_samples)
}

df = pd.DataFrame(data)

In [ ]:
df['d_min'] = df[['d_f', 'd_b', 'd_r', 'd_l']].min(axis=1)
df['reverse_light'] = (df['v'] < 0).astype(int)  #Hız negatif ise araç geri gidiyordur. Geri viteste lamba yanar
df['headlight'] = np.where(df['r'] > 0.6, 1, 0) # Yağmur şiddetliyse far açılır.
df['buzzer'] = (df['d_min'] < 0.2).astype(int) # Dar alan: 20cm altında buzzer çalışır.
df['brake_light'] = (df['v'] < 0.1).astype(int)  #Hız durma noktasına (0.1 m/s) gelmişse fren lambası yanar.
df['yaw_rate'] = (df['v'] / L) * np.tan(df['δ'])
df['feasible'] = int(W_slot - w_car > s_margin)
df['slip_risk'] = 1 / (1 + np.exp(-( -4 + 7*df['r'] + 3*np.abs(df['v']) )))
df['delta_target'] = np.clip(k_theta * df['e_θ'] + k_y * df['e_lane'], -delta_max, delta_max)
df['d_brake'] = (df['v']**2) / (2 * a_base * (1 - 0.4 * df['r']))
angle_risk = (np.abs(df['delta_target']) > 0.3) & (df['d_min'] < 0.3)
df['score'] = (
    (df['d_min'] > 0.2).astype(int) * 40 +
    (np.abs(df['e_θ']) < 0.1).astype(int) * 35 +
    (df['slip_risk'] < 0.3).astype(int) * 25
)


# Label: 0 = Başarılı, 1 = Riskli
# Dar slotta hız 1.0 m/s üzerindeyse veya mesafe 15cm altındaysa direkt risklidir.
df['label'] = (
    (df['score'] < 70) |
    (df['d_min'] < 0.15) |
    (df['v'] > 1.0) |
    (angle_risk)
).astype(int)
df[['d_f', 'd_b', 'd_r', 'd_l']] += np.random.normal(0, 0.015, (num_samples, 4)) #gürültü eklendi

In [ ]:
df

,id,r,d_f,d_b,d_r,d_l,v,e_lane,conf_lane,batt,...,headlight,buzzer,brake_light,yaw_rate,feasible,slip_risk,delta_target,d_brake,score,label
0,1,0.638145,0.400909,0.619142,0.440441,0.272484,0.293878,-0.051904,0.870842,0.650053,...,1,0,0,-0.011074,1,0.793908,0.029182,0.014496,75,0
1,2,0.459292,0.538016,0.417129,0.107145,0.561300,1.170129,0.232219,0.774782,0.619686,...,0,1,0,0.147554,1,0.938511,-0.040359,0.209670,0,1
2,3,0.964499,0.405750,0.538275,0.229711,0.458197,1.489878,0.079699,0.718887,0.543756,...,1,0,0,0.009032,1,0.999270,-0.007795,0.451753,75,1
3,4,0.218978,0.143275,0.836964,0.342684,0.404925,1.213371,0.042990,0.868403,0.394690,...,0,1,0,-0.073270,1,0.763680,0.080557,0.201701,0,1
4,5,0.587856,0.210610,0.807878,0.519304,0.157180,0.323057,-0.148713,0.766358,0.693008,...,0,1,0,-0.043398,1,0.747282,0.007569,0.017056,35,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14996,0.193418,0.709063,0.325790,0.270935,0.452741,1.439053,0.057720,0.936656,0.195472,...,0,0,0,0.155660,1,0.841721,0.108306,0.280566,40,1
14996,14997,0.319452,0.542365,0.438386,0.542268,0.208743,0.371832,-0.171314,0.938850,0.664160,...,0,1,0,-0.055509,1,0.343363,-0.118840,0.019814,0,1
14997,14998,0.353408,0.122922,0.876088,0.571103,0.119304,0.248415,-0.171256,0.796616,0.326906,...,0,1,0,0.026313,1,0.314126,0.025722,0.008984,0,1
14998,14999,0.286822,0.345102,0.667384,0.250168,0.428400,0.433253,0.068471,0.807277,0.229328,...,0,0,0,-0.021846,1,0.333480,0.007841,0.026504,75,0


In [ ]:
df.to_csv('otonom_park_dataset.csv', index=False, encoding='utf-8')

print("Veri seti 'otonom_park_dataset.csv' başarıyla kaydedildi!")

Veri seti 'otonom_park_dataset.csv' başarıyla kaydedildi!
